In [ ]:
import scipy.io as sio
import pandas as pd
import numpy as np

In [ ]:
description = pd.read_excel('description.xlsx')

description.head()

In [ ]:
import gensim
from gensim.models.doc2vec import Word2Vec
import gensim.downloader as api
import nltk
# nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
print(list(api.info()['models'].keys()))

In [ ]:
wv = api.load('word2vec-google-news-300')

In [ ]:
# wv.save('fasttext-wiki-news-subwords-300.kv')

In [ ]:
# wv = gensim.models.KeyedVectors.load('fasttext-wiki-news-subwords-300.kv')

In [ ]:
def preprocess_text(text):
    text = text.lower()

    words = gensim.utils.simple_preprocess(text)

    stop_words = gensim.parsing.preprocessing.STOPWORDS
    words = [word for word in words if word not in stop_words]

    return words

In [ ]:
def get_sentence_embedding(sentence):
    words = preprocess_text(sentence)

    word_embeddings = [wv[word] for word in words if word in wv]

    sentence_embedding = np.mean(word_embeddings, axis=0)

    return sentence_embedding

In [ ]:
def get_paragraph_embedding(paragraph):
    sentences = nltk.sent_tokenize(paragraph)

    sentence_embeddings = [get_sentence_embedding(sentence) for sentence in sentences]

    paragraph_embedding = np.mean(sentence_embeddings, axis=0)

    return paragraph_embedding

In [ ]:
paragraph = "This is a sample paragraph. It contains multiple sentences."
paragraph_embedding = get_paragraph_embedding(paragraph)

In [ ]:
paragraph_embedding.shape

In [ ]:
paragraph_embedding

In [ ]:
all_cls_org = []

for i in range(len(description['label'])):
    all_cls_org.append(description['label'][i])

all_cls_org

In [ ]:
att_mat = sio.loadmat('att_splits.mat')

In [ ]:
description = description.reset_index()
description.head()

In [ ]:
paragraphs = description['description'].tolist()

In [ ]:
paragraphs

In [ ]:
embedding = []

for c in all_cls_org:
    for index, row in description.iterrows():
        if row['label'] == c:
            print('matched: ' , row['label'], c)
            text = row['description']
            embedding.append(get_paragraph_embedding(text))



In [ ]:
embedding_array = np.vstack(embedding)

In [ ]:
embedding_array.shape

In [ ]:
embedding_array

In [ ]:
print("Min:", embedding_array.min(), ", Max:", embedding_array.max())

In [ ]:
# from sklearn import preprocessing

# min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 100))

In [ ]:
# embedding_array = min_max_scaler.fit_transform(embedding_array)

In [ ]:
embedding_array.shape

In [ ]:
print("Min:", embedding_array.min(), ", Max:", embedding_array.max())

In [ ]:
att_mat['att'].shape

In [ ]:
att_mat['att'] = embedding_array.T

In [ ]:
att_mat['att'].shape

In [ ]:
sio.savemat('word2vec_cls_description.mat', att_mat)